A continuación, implementaremos una clase `ClassificationRequest `que garantiza la consistencia de los datos de entrada (para predicción), asegurando que mantengan la misma estructura y tipos de datos utilizados durante el entrenamiento del modelo de clasificación.

Lo tipos de datos que se usaron en el entrenamiento fueron los siguientes:

- historial_pagos_atrasados: int64  
- calificacion_buro: int64  
- monto_solicitado_mxn: float64
- ratio_deuda_ingresos: float64
- carga_total_ingresos: float64

El método `from_input` permite convertir  convertir múltiples tipos de entrada (dict, lista, CSV, DataFrame) en un dataframe limpio y consistente con las columnas requeridas por el modelo. El nombre de estas columnas se obtienen de los campos definidos en la clase, al utilizar el código `required_fields = list(cls.model_fields.keys())`. Así mismo comprueba si hacen falta columnas.


In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, Union
import pandas as pd
import numpy as np
from io import StringIO

class ClassificationRequest(BaseModel):
    """
    Modelo de datos para solicitudes de clasificación crediticia.

    Esta clase define la estructura esperada de la información de entrada
    para un modelo de clasificación, utilizando validación de tipos y
    descripciones semánticas a través de Pydantic. Cada campo representa
    una variable relevante para evaluar el riesgo crediticio del cliente.

    Attributes
    ----------
    historial_pagos_atrasados : Optional[int]
        Número de pagos atrasados del cliente.
    calificacion_buro : Optional[int]
        Puntaje del buró de crédito del cliente.
    monto_solicitado_mxn : Optional[float]
        Monto del préstamo solicitado en pesos mexicanos.
    ratio_deuda_ingresos : Optional[float]
        Relación entre la deuda total del cliente y sus ingresos.
    carga_total_ingresos : Optional[float]
        Porcentaje de carga total sobre los ingresos del cliente.

    Métodos
    -------
    from_input(data)
        Convierte diferentes tipos de entrada (DataFrame, diccionario, lista o CSV)
        en un DataFrame con las columnas requeridas por el modelo.
    """
    historial_pagos_atrasados: Optional[int] = Field(None, description="Número de pagos atrasados del cliente")
    calificacion_buro: Optional[int] = Field(None, description="Puntaje del buró de crédito")
    monto_solicitado_mxn: Optional[float] = Field(None, description="Monto del préstamo solicitado en pesos")
    ratio_deuda_ingresos: Optional[float] = Field(None, description="Relación entre deuda total e ingresos")
    carga_total_ingresos: Optional[float] = Field(None, description="Porcentaje de carga total sobre los ingresos")

    @classmethod
    def from_input(cls, data: Union[str, pd.DataFrame, dict, list, bytes]):
        """
        Convierte una entrada de datos en un DataFrame validado con las columnas del modelo.

        Este método acepta múltiples formatos de entrada —como DataFrame, diccionario,
        lista de registros o archivo CSV (en texto o bytes)— y los transforma en un
        `pandas.DataFrame` con las columnas definidas en la clase. Si faltan columnas
        requeridas o el formato no es soportado, se lanza una excepción descriptiva.

        Parameters
        ----------
        data : Union[str, pd.DataFrame, dict, list, bytes]
            Datos de entrada que representan uno o varios registros. Puede ser:
            - `pd.DataFrame`: datos ya tabulares.
            - `dict`: un solo registro o un diccionario con listas de valores.
            - `list`: lista de diccionarios (cada uno una fila).
            - `str` o `bytes`: contenido de un archivo CSV.

        Return
        -------
        pd.DataFrame
            DataFrame con las columnas requeridas por el modelo `ClassificationRequest`.
        
        Raises
        ------
        ValueError
            Si no se puede leer el archivo csv.
            Si faltan las columnas requeridas
        TypeError
            Si el tipo de datos no es soportado.
        """
        if isinstance(data, pd.DataFrame):
            df = data.copy()

        elif isinstance(data, dict):
            try:
                # Convierte cualquier dict con listas (incluso de distinto largo) a DataFrame
                df = pd.DataFrame.from_dict(data, orient="index").T
            except Exception:
                # Si no son listas (o falla), tratamos como un solo registro
                df = pd.DataFrame([data])

        elif isinstance(data, list):
            df = pd.DataFrame(data)

        elif isinstance(data, (str, bytes)):
            try:
                df = pd.read_csv(StringIO(data))
            except Exception as e:
                raise ValueError(f"No se pudo leer el CSV: {e}")
        else:
            raise TypeError("El tipo de entrada no es soportado. Debe ser DataFrame, dict, lista o CSV.")

        required_fields = list(cls.model_fields.keys())
        missing_columns = [f for f in required_fields if f not in df.columns]
        if missing_columns:
            raise ValueError(
                f"Faltan columnas requeridas: {missing_columns}. "
                f"Columnas disponibles: {list(df.columns)}"
            )
        
        df = df[required_fields]
        type_map = {
            "historial_pagos_atrasados": "Int64",  # Entero nulo seguro
            "calificacion_buro": "Int64",
            "monto_solicitado_mxn": "float64",
            "ratio_deuda_ingresos": "float64",
            "carga_total_ingresos": "float64",
        }
        try:
            df = df.astype(type_map)
        except Exception as e:
            raise ValueError(f"No se pudieron convertir los tipos correctamente: {e}")
        return df


Probemos la clase

In [12]:
data_correcto = {
    "historial_pagos_atrasados": [2, 0, 4],
    "calificacion_buro": [650, 700, 900],
    "monto_solicitado_mxn": [50000.0, 30000.0, 90.0],
    "ratio_deuda_ingresos": [np.nan, 0.2, 0.1],
    "carga_total_ingresos": [0.3, 0.1]
}

# Datos con columnas faltantes
data_faltante = {
    "historial_pagos_atrasados": [2],
    "calificacion_buro": [650]
}

In [13]:
df_correcto = ClassificationRequest.from_input(data_correcto)
df_correcto.head()

,historial_pagos_atrasados,calificacion_buro,monto_solicitado_mxn,ratio_deuda_ingresos,carga_total_ingresos
0,2,650,50000.0,NaN,0.3
1,0,700,30000.0,0.2,0.1
2,4,900,90.0,0.1,NaN


Veamos con un archivo csv

In [14]:
archivo_csv = "/media/luisgarcia/Datos/32. Ejercicio de acercamiento al rol/mlops_pyme/data/processed/covalto_sme_credit_test.csv"
df = ClassificationRequest.from_input(archivo_csv)
df.head()

,historial_pagos_atrasados,calificacion_buro,monto_solicitado_mxn,ratio_deuda_ingresos,carga_total_ingresos
0,1,3,100790.96,0.722558,0.739974
1,1,3,616564.64,0.840515,1.061708
2,1,2,177995.46,0.623165,1.070324
3,3,0,91935.97,NaN,NaN
4,1,3,1006577.05,0.756672,0.948822


In [15]:
data ={
    "historial_pagos_atrasados": [0, 1, 1, 3, 1],
    "calificacion_buro": [1, 3, 2, 0, 3],
    "monto_solicitado_mxn": [100790.96, 616564.64, 177995.46, 91935.97, 1006577.05],
    "ratio_deuda_ingresos": [0.722558, 0.840515, 0.623165, np.nan, 0.756672],
    "carga_total_ingresos": [0.739974, 1.061708, 1.070324, np.nan, 0.948822]
}
df = ClassificationRequest.from_input(data)
df.head()

,historial_pagos_atrasados,calificacion_buro,monto_solicitado_mxn,ratio_deuda_ingresos,carga_total_ingresos
0,0,1,100790.96,0.722558,0.739974
1,1,3,616564.64,0.840515,1.061708
2,1,2,177995.46,0.623165,1.070324
3,3,0,91935.97,NaN,NaN
4,1,3,1006577.05,0.756672,0.948822
